# ngl::Util

This header file includes a number of free functions for doing useful 3D graphics operations. The following demos explain what the functions do and how to use them.

In [1]:
// You will need to modify these paths to suit
#pragma cling add_library_path("/Users/jmacey/NGL/lib")
#pragma cling load("/Users/jmacey/NGL/lib/libNGL.dylib")
#pragma cling add_include_path("/Users/jmacey/NGL/include")


In [2]:
#include <ngl/Util.h>
#include <ngl/Mat4.h>
#include <ngl/NGLStream.h> // for printing
#include <iostream>


## Degrees to Radians conversion

It's important to remember that most cmath functions take radians as the default arguments to things like ```sin``` and ```cos``` ngl implements functions similar to python to do the conversions.

In [8]:
{
    ngl::Real rad=ngl::radians(45.0f);
    ngl::Real deg=ngl::degrees(rad);
    std::cout<<"rad is "<<rad<<" deg "<<deg<<'\n';
    
}

rad is 0.785398 deg 45


## ngl::lookAt

```ngl::lookAt``` takes three ```ngl::Vec3``` as parameters to specify the  ```eye```, ```look``` and ```up``` positions. It will then return an ```ngl::Mat4``` with the transformations required to generate the scale, rotate and translations required for the objects vertices to be transformed by.

To do this we calculate the following vectors :-
$$
\mathbf{n} = \mathbf{eye} - \mathbf{look} \\\
\mathbf{u} = \mathbf{up} \times \mathbf{n} \\\
\mathbf{v} = \mathbf{n} \times \mathbf{u}
$$

Then normalize to unit length.

The job of the View matrix is to then convert world co-ordinates to camera co-ordinates it must transform the camera's coordinate system into the generic position for the camera. 

$$
V=\begin{pmatrix}
v_y & v_y & v_z & d_x \\\
u_y & u_y & u_z & d_y \\\
n_y & n_y & n_z & d_z \\\
0 & 0 & 0 & 1 
\end{pmatrix}
$$

$$
(d_x,d_y,d_z)= ( -\mathbf{eye} \bullet \mathbf{u} , -\mathbf{eye} \bullet \mathbf{v} , -\mathbf{eye} \bullet \mathbf{n})
$$


In [9]:
{
    ngl:: Vec3 eye{2.0f,2.0f,2.0f};
    ngl::Vec3 look{0.0f,0.0f,0.0f};
    ngl::Vec3 up{0.0f,1.0f,0.0f};
    auto lookNGL=ngl::lookAt(eye,look,up);
    std::cout<<lookNGL<<'\n';
}

[0.707107,0,-0.707107,-0]
[-0.408248,0.816497,-0.408248,-0]
[0.57735,0.57735,0.57735,-3.4641]
[0,0,0,1]



## [Orthographic Projection](https://en.wikipedia.org/wiki/Orthographic_projection)
An orthographic projection is specified by setting “clip” planes. We do this by calculating the following planes ```near - far``` ```left - right``` and  ```top - bottom```  from this we can then create a matrix to scale and translate our vertices to make them fit into the canonical view volume ( a unit cube with a center at $[0,0,0]$ )

$$
V=\begin{pmatrix}
\frac{2.0}{r-l} & 0 & 0 & - \frac{r+l}{r-l} \\\
0 & \frac{2.0}{t-b} & 0 & -\frac{t+b}{t-b} \\\
0 & 0 & \frac{2.0}{f-n} & -\frac{f+n}{f-n} \\\
0 & 0 & 0 & 1 
\end{pmatrix}
$$


In [11]:
{
    ngl::Mat4 orth=ngl::ortho(-180.0f, 180.0f, -150.0f,150.0f,1.0f,-1.0f);
    std::cout<<"Orthographic matrix \n"<<orth<<'\n';
}

Orthographic matrix 
[0.00555556,0,0,-0]
[0,0.00666667,0,-0]
[0,0,1,0]
[0,0,0,1]



There is also a 2d variant that clips the z component to -1 1

In [13]:
{
    auto orth= ngl::ortho(-10.0f, 10.0f, -5.0f,5.0f) ;
    std::cout<<"2d ortho \n"<<orth<<'\n';
}

2d ortho 
[0.1,0,0,-0]
[0,0.2,0,-0]
[0,0,-1,0]
[0,0,0,1]



## [Perspective Projection](https://en.wikipedia.org/wiki/3D_projection)

Perspective projection generates a matrix  
